# FYS - STK Project 2

## Classification with logistic regression

In [17]:
#Importing and showing the data
pandas as pd
from sklearn.datasets import load_breast_cancer
%matplotlib inline
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = pd.Series(data.target)
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


The data set consists of 569 measurements with 30 features, labeled in the last column shown above, as benign or malignant. This data set will be used to train two classification algorithms: Logistic regression with gradient descent and a neural network with stochastic gradient descent. This section develops the logistic regression algorithm. The data set is split into a test and train set using the sklearn method. 

In [28]:
from sklearn.model_selection import train_test_split
features, target = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(features, target)
X_train.shape, y_train.shape

((426, 30), (426,))

Using just the raw data without any preprocessing:

In [30]:
from LogReg import LogisticRegression
import time
t0 = time.time()
w_GD = LogisticRegression(X_train, y_train,l_rate = 1e-7,
                     tol=1e-1, max_iter = 1e+7)
t = time.time() - t0
t

373.83637499809265

In [32]:
t/60

6.23060624996821

The algorithm uses over six minutes! The output of the logistic regression is an array of coefficients $\pmb{\beta}$ where 
$$ \pmb{p} = \frac{\exp{\pmb{X \beta}}}{1 + \exp{\pmb{X \beta}}}$$
is the probability of the sample being in the target class. Below, the weights are applied to the test set feature matrix, and compared to the labels.

In [40]:
from LogReg import sigmoid

def test_pred(X_test, weights, y_test):
    pred = sigmoid(np.dot(X_test, weights))
    for i in range(len(pred)):
        if pred[i] > 0.5:
            pred[i] = 1
        else:
            pred[i] = 0
    s=0
    for i in range(38):
        if pred[i] == y_test[i]:
            s += 1
    return s/38


The classifier predicts almost 95 \%  of the test samples correctly. However, it is extremely slow. To reduce the amount of data, a PCA is applied to pick out a projection onto orthogonal subspaces where 95\% of the variance is retained in the feature set. The features are also scaled to unit variance and the mean is substracted to hopefully improve the amount of correct classifications.

In [38]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.decomposition import PCA
pca= PCA(n_components=.95)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
X_train.shape

(426, 11)

Applying logistic regression again:

In [36]:
t0 = time.time()
w_GD = LogisticRegression(X_train, y_train,l_rate = 1e-7,
                     tol=1e-1, max_iter = 1e+7)
t = time.time() - t0

In [37]:
t/60

5.916258001327515

In [41]:
test_pred(X_test, w_GD, y_test)

0.47368421052631576